In [1]:
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision import models
from torchsummary import summary
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import urllib
from matplotlib import cm
import cv2
from utils import *

# update the requirements.txt file
# !pip freeze > requirements.txt

/home/pradeep/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [4]:
weights = models.segmentation.DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1
weights_pretrained = models.ResNet101_Weights.IMAGENET1K_V2
model = models.segmentation.deeplabv3.deeplabv3_resnet101(weights=weights, weights_backbone=weights_pretrained)

In [8]:
image = cv2.imread('Data/1.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = preprocess_image(image)

model.eval()
output = model(image.unsqueeze(0))

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [17]:
import pickle

with open('segmented_images.pkl', 'rb') as f:
    data_dict = pickle.load(f)

In [18]:
loss_list = []

for file_name, label in data_dict.items(): 
    image = cv2.imread(file_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    image = preprocess_image(image)
    model.eval()
    output = model(image.unsqueeze(0))
    
    output = output['out']
    
    # train with ground truth as label
    label = torch.tensor(label)
    label = label.unsqueeze(0)
    
    loss = F.cross_entropy(output, label)
    print(loss.item())
    loss_list.append(loss.item())
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    # save the model
    torch.save(model.state_dict(), 'model.pth')
    